In [ ]:
!pip3 install torch torchvision torchaudio

In [100]:
import numpy as np
import torch
from datetime import datetime, timedelta
from torch.nn import Linear, ModuleList, Parameter
from dyrep_node_Hawkes import DyRepNode
from earthquake_data_loader import EarthquakeDataset
from torch.utils.data import DataLoader
from train_eval_main_node import get_return_time
import copy
import importlib
import time
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm
from sklearn.metrics import average_precision_score, roc_auc_score
from utils import *
rnd = np.random.RandomState(42)

In [73]:
# dyrep_node_Hawkes 모듈을 다시 로드합니다.
importlib.reload(dyrep_node_Hawkes)
from dyrep_node_Hawkes import DyRepNode
model= DyRepNode(num_nodes= 100,
                  hidden_dim= 4,
                  random_state= rnd, 
                  first_date=1, 
                  end_datetime=1, 
                  num_neg_samples= 2, 
                  num_time_samples = 5,
                 device= 'mps', all_comms=False, train_td_max=None)


In [83]:
params_main = [param for param in model.parameters() if param.requires_grad]
optimizer = optim.Adam(params_main, lr=0.0002, betas=(0.5, 0.999))
scheduler = lr_scheduler.MultiStepLR(optimizer, [20], gamma=0.5)

In [95]:
node_degree=np.zeros((100, 1))
node_embeddings_initial= np.tile([0,0,0,0], (100, 1))
train_data = EarthquakeDataset("train")
time_bar_initial = np.zeros((train_data.N_nodes, 1)) + train_data.FIRST_DATE.timestamp()
time_bar = copy.deepcopy(time_bar_initial)
train_loader = DataLoader(train_data,batch_size=1, shuffle=False)
A=train_data.get_Adjacency()
node_degree = np.sum(A, axis=0)
model.reset_state(node_embeddings_initial=np.tile([0,0,0,0], (100, 1)),
                    A_initial=A,
                    node_degree_initial=node_degree,
                    time_bar=time_bar)
model.train()
train_loader.dataset.time_bar = time_bar


A_initial 3478.0


In [97]:
start = time.time()
total_loss = 0
total_loss_lambda, total_loss_surv = 0, 0

In [102]:
for batch_idx, data_batch in enumerate(tqdm(train_loader)):
            # backprop 전에 기울기를 초기화
            optimizer.zero_grad()
            # 데이터 type 확정하고, gpu에 올려놓기
            data_batch[1] = data_batch[2].float().to('mps')
            data_batch[2] = data_batch[4].double().to('mps')
            data_batch[3] = data_batch[5].double()# no need of GPU
            
            # 모델 forward 돌리고 역전파 생성
            # 단순히 log(pos)- neg 형태의 기울기 반영
            output = model(data_batch)
            losses = [-torch.sum(torch.log(output[0]) + 1e-10), torch.sum(output[1])]
            loss = torch.sum(torch.stack(losses))/args.batch_size
            loss.backward()

  0%|          | 0/49402 [00:00<?, ?it/s]


AttributeError: 'numpy.ndarray' object has no attribute 'toordinal'